In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [6]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/models/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Mounted at /content/drive


['res10_300x300_ssd_iter_140000_fp16.caffemodel',
 'deploy.prototxt.txt',
 'dlib_face_recognition_resnet_model_v1.dat',
 'mmod_human_face_detector.dat',
 'shape_predictor_5_face_landmarks.dat',
 'shape_predictor_68_face_landmarks.dat',
 'openface.nn4.small2.v1.t7']

In [7]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [8]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.07813147 -0.02060156 -0.0401138  ...  0.10667205  0.01939723
    0.04673895]]

 [[ 0.05185256  0.00279914 -0.0644751  ...  0.01517338  0.01929869
    0.07129709]]

 [[-0.08312224  0.15255336 -0.00542035 ...  0.10915315  0.10894079
    0.13349088]]

 ...

 [[-0.00476215  0.10683095  0.04353967 ...  0.08243319  0.07582808
    0.00351276]]

 [[ 0.04948314  0.18561181  0.0562551  ... -0.0060073   0.12891996
   -0.04651411]]

 [[-0.05880731  0.24519315  0.09106579 ... -0.11058079  0.19247277
   -0.06857494]]]
['1721030456_Đào Ngọc Huy' '1721030456_Đào Ngọc Huy'
 '1721030456_Đào Ngọc Huy' ... '1721030453_NguyenVuThanhLong'
 '1721030453_NguyenVuThanhLong' '1721030453_NguyenVuThanhLong']


In [9]:
x.shape

(4694, 1, 128)

In [10]:
y.shape

(4694,)

In [11]:
x = x.reshape(-1, 128)

In [12]:
x.shape

(4694, 128)

In [13]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [14]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3755, 128), (939, 128), (3755,), (939,))

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [16]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [17]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [18]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.72
Accurency Test = 0.66
F1 Score Train = 0.50
F1 Score Test = 0.47


In [19]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [20]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.83
F1 Score Train = 0.82
F1 Score Test = 0.72


In [21]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [22]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [23]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [24]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.83
F1 Score Train = 0.97
F1 Score Test = 0.74


In [25]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [26]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  10.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=  11.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.6s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [27]:
model_best_estimator = model_grid.best_estimator_

In [28]:
model_grid.best_score_

0.8399442320478016

In [29]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))